In [203]:

import random
import requests
import os
from dotenv import load_dotenv
import time
import datetime
import warnings
warnings.filterwarnings('ignore')
import pandas as pd


**Processo para gerar dados de local do cliente**

In [204]:

# Carregar variáveis de ambiente (API Key)
load_dotenv()
API_KEY = os.getenv('HERE_API_KEY')


In [205]:
API_KEY

'YW0MtjazNFSgTvqPE0oMO5bEdQLB_bPWIqzoJQdtIMs'

In [206]:
# Função para obter latitude e longitude do endereço usando a API Here
def geocode_address(address):
    url = f'https://geocode.search.hereapi.com/v1/geocode'
    params = {
        'q': address,
        'apiKey': API_KEY
    }
    response = requests.get(url, params=params)
    
    # Verificar o status da resposta
    if response.status_code == 200:
        data = response.json()
        if data['items']:
            latitude = data['items'][0]['position']['lat']
            longitude = data['items'][0]['position']['lng']
            return latitude, longitude
    return None, None  # Retorna None se não encontrar as coordenadas



**estrutura da tabela de clientes**

***Tabela cliente***

nome, sobrenome, cep_residencia, estado_residencia, cidade_residencia, bairro_residencia, rua_residencia, dias_preferencia, turno_preferencia, horario_preferencia, cep_preferencia, estado_preferencia, cidade_preferencia, bairro_preferencia, rua_preferencia

***Tabela Clinica***

nome, cnpj, cep, estado, cidade, bairro, rua, nota_pesquisa, valor_consulta

In [207]:
import pandas as pd

df_clientes = pd.DataFrame({
    "nome": ["João", "Maria", "Carlos", "Ana", "Pedro", "Juliana", "Lucas", "Mariana", "Rafael", "Fernanda","Claudio"],
    "sobrenome": ["Silva", "Oliveira", "Santos", "Souza", "Pereira", "Costa", "Lima", "Mendes", "Almeida", "Ferreira","Bispo"],
    "cep_preferencia": ["05868-130", "04773-110", "05860-020", "04773-120", "05868-140", 
            "05860-030", "05868-150", "05860-040", "04773-130", "05868-150","05763-290"]
})

df_clientes


,nome,sobrenome,cep_preferencia
0,João,Silva,05868-130
1,Maria,Oliveira,04773-110
2,Carlos,Santos,05860-020
3,Ana,Souza,04773-120
4,Pedro,Pereira,05868-140
5,Juliana,Costa,05860-030
6,Lucas,Lima,05868-150
7,Mariana,Mendes,05860-040
8,Rafael,Almeida,04773-130
9,Fernanda,Ferreira,05868-150


In [208]:
# Criar uma coluna como ID/Indice para facilitar a simulação no banco de dados
df_clientes['ID_Cliente'] = range(1, len(df_clientes) + 1)
df_clientes

,nome,sobrenome,cep_preferencia,ID_Cliente
0,João,Silva,05868-130,1
1,Maria,Oliveira,04773-110,2
2,Carlos,Santos,05860-020,3
3,Ana,Souza,04773-120,4
4,Pedro,Pereira,05868-140,5
5,Juliana,Costa,05860-030,6
6,Lucas,Lima,05868-150,7
7,Mariana,Mendes,05860-040,8
8,Rafael,Almeida,04773-130,9
9,Fernanda,Ferreira,05868-150,10


In [209]:
df_clientes = df_clientes[['ID_Cliente','nome','sobrenome','cep_preferencia']]
df_clientes

,ID_Cliente,nome,sobrenome,cep_preferencia
0,1,João,Silva,05868-130
1,2,Maria,Oliveira,04773-110
2,3,Carlos,Santos,05860-020
3,4,Ana,Souza,04773-120
4,5,Pedro,Pereira,05868-140
5,6,Juliana,Costa,05860-030
6,7,Lucas,Lima,05868-150
7,8,Mariana,Mendes,05860-040
8,9,Rafael,Almeida,04773-130
9,10,Fernanda,Ferreira,05868-150


In [210]:
# Obter latitude e longitude para cada endereço
coordenadas = df_clientes["cep_preferencia"].apply(geocode_address)
df_clientes["latitude"], df_clientes["longitude"] = zip(*coordenadas)
df_clientes

,ID_Cliente,nome,sobrenome,cep_preferencia,latitude,longitude
0,1,João,Silva,05868-130,-23.67145,-46.78343
1,2,Maria,Oliveira,04773-110,-23.68298,-46.70859
2,3,Carlos,Santos,05860-020,-23.66574,-46.76681
3,4,Ana,Souza,04773-120,-23.68342,-46.70933
4,5,Pedro,Pereira,05868-140,-23.67058,-46.78307
5,6,Juliana,Costa,05860-030,-23.66329,-46.76797
6,7,Lucas,Lima,05868-150,-23.67019,-46.78201
7,8,Mariana,Mendes,05860-040,-23.66592,-46.76628
8,9,Rafael,Almeida,04773-130,29.03238,-82.19994
9,10,Fernanda,Ferreira,05868-150,-23.67019,-46.78201


**Processo para gerar os dados das clinicas**

In [211]:

# Dados fixos das clínicas
clinicas_data = [
    {"nome": "Clínica Saúde e Bem Estar", "cep": "05868-130", "nota_pesquisa": 4.2, "valor_consulta": 150},
    {"nome": "Clínica Médico e Família", "cep": "04773-110", "nota_pesquisa": 3.8, "valor_consulta": 120},
    {"nome": "Clínica Boa Vista", "cep": "05860-020", "nota_pesquisa": 4.5, "valor_consulta": 180},
    {"nome": "Clínica Estrela da Saúde", "cep": "04773-120", "nota_pesquisa": 4.0, "valor_consulta": 170},
    {"nome": "Clínica Vida Plena", "cep": "05868-140", "nota_pesquisa": 3.7, "valor_consulta": 160},
    {"nome": "Clínica Sol e Saúde", "cep": "05860-030", "nota_pesquisa": 4.3, "valor_consulta": 190},
    {"nome": "Clínica Nova Vida", "cep": "05868-150", "nota_pesquisa": 4.6, "valor_consulta": 200},
    {"nome": "Clínica São Paulo Saúde", "cep": "05860-040", "nota_pesquisa": 4.1, "valor_consulta": 150},
    {"nome": "Clínica Médica São José", "cep": "04773-130", "nota_pesquisa": 3.9, "valor_consulta": 130},
    {"nome": "Clínica Esperança", "cep": "05868-150", "nota_pesquisa": 4.0, "valor_consulta": 160},
    {"nome": "Clínica Saúde Vida", "cep": "04773-140", "nota_pesquisa": 4.2, "valor_consulta": 175},
    {"nome": "Clínica da Família", "cep": "05868-160", "nota_pesquisa": 3.6, "valor_consulta": 140},
    {"nome": "Clínica Médica Sul", "cep": "05860-050", "nota_pesquisa": 4.4, "valor_consulta": 180},
    {"nome": "Clínica Mais Saúde", "cep": "05868-170", "nota_pesquisa": 4.5, "valor_consulta": 210},
    {"nome": "Clínica Equilíbrio", "cep": "04773-150", "nota_pesquisa": 4.0, "valor_consulta": 165},
    {"nome": "Clínica Vida Nova", "cep": "05860-060", "nota_pesquisa": 4.1, "valor_consulta": 155},
    {"nome": "Clínica Paulista", "cep": "05868-180", "nota_pesquisa": 3.8, "valor_consulta": 145},
    {"nome": "Clínica Bem Estar", "cep": "04773-160", "nota_pesquisa": 4.2, "valor_consulta": 150},
    {"nome": "Clínica Médica São Paulo", "cep": "05860-070", "nota_pesquisa": 3.7, "valor_consulta": 130},
    {"nome": "Clínica Saúde de Ouro", "cep": "05868-190", "nota_pesquisa": 4.3, "valor_consulta": 190},
    {"nome": "Clínica Viver Bem", "cep": "04773-170", "nota_pesquisa": 4.4, "valor_consulta": 200},
    {"nome": "Clínica Equilibrio Sul", "cep": "05860-080", "nota_pesquisa": 3.9, "valor_consulta": 165},
    {"nome": "Clínica São Marcos", "cep": "05868-200", "nota_pesquisa": 4.1, "valor_consulta": 160},
    {"nome": "Clínica Dor e Bem Estar", "cep": "04773-180", "nota_pesquisa": 3.6, "valor_consulta": 140},
    {"nome": "Clínica Saúde Melhor", "cep": "05860-090", "nota_pesquisa": 4.2, "valor_consulta": 175},
    {"nome": "Clínica Saúde Total", "cep": "05868-210", "nota_pesquisa": 4.5, "valor_consulta": 200},
    {"nome": "Clínica Médica Eficaz", "cep": "04773-190", "nota_pesquisa": 4.3, "valor_consulta": 180},
    {"nome": "Clínica Luz e Vida", "cep": "05860-100", "nota_pesquisa": 4.0, "valor_consulta": 150},
    {"nome": "Clínica Aconchego", "cep": "05868-220", "nota_pesquisa": 3.8, "valor_consulta": 145},
    {"nome": "Clínica Bem Estar Sul", "cep": "04773-200", "nota_pesquisa": 4.2, "valor_consulta": 160},
    {"nome": "Clínica Saúde Sempre", "cep": "05860-110", "nota_pesquisa": 4.3, "valor_consulta": 190}
]

# Gerar o DataFrame com as clínicas
df_clinicas = pd.DataFrame(clinicas_data)

# Exibir os dados
df_clinicas.head()


,nome,cep,nota_pesquisa,valor_consulta
0,Clínica Saúde e Bem Estar,05868-130,4.2,150
1,Clínica Médico e Família,04773-110,3.8,120
2,Clínica Boa Vista,05860-020,4.5,180
3,Clínica Estrela da Saúde,04773-120,4.0,170
4,Clínica Vida Plena,05868-140,3.7,160


In [212]:
# Criar uma coluna como ID/Indice para facilitar a simulação no banco de dados
df_clinicas['ID_Clinica'] = range(1, len(df_clinicas) + 1)
df_clinicas.head()

,nome,cep,nota_pesquisa,valor_consulta,ID_Clinica
0,Clínica Saúde e Bem Estar,05868-130,4.2,150,1
1,Clínica Médico e Família,04773-110,3.8,120,2
2,Clínica Boa Vista,05860-020,4.5,180,3
3,Clínica Estrela da Saúde,04773-120,4.0,170,4
4,Clínica Vida Plena,05868-140,3.7,160,5


In [213]:
df_clinicas = df_clinicas[['ID_Clinica','nome','cep','nota_pesquisa','valor_consulta']]
df_clinicas.head()

,ID_Clinica,nome,cep,nota_pesquisa,valor_consulta
0,1,Clínica Saúde e Bem Estar,05868-130,4.2,150
1,2,Clínica Médico e Família,04773-110,3.8,120
2,3,Clínica Boa Vista,05860-020,4.5,180
3,4,Clínica Estrela da Saúde,04773-120,4.0,170
4,5,Clínica Vida Plena,05868-140,3.7,160


In [214]:
# Obter latitude e longitude para cada endereço
coordenadas = df_clinicas["cep"].apply(geocode_address)
df_clinicas["latitude"], df_clinicas["longitude"] = zip(*coordenadas)


In [215]:
df_clinicas.head()

,ID_Clinica,nome,cep,nota_pesquisa,valor_consulta,latitude,longitude
0,1,Clínica Saúde e Bem Estar,05868-130,4.2,150,-23.67145,-46.78343
1,2,Clínica Médico e Família,04773-110,3.8,120,-23.68298,-46.70859
2,3,Clínica Boa Vista,05860-020,4.5,180,-23.66574,-46.76681
3,4,Clínica Estrela da Saúde,04773-120,4.0,170,-23.68342,-46.70933
4,5,Clínica Vida Plena,05868-140,3.7,160,-23.67058,-46.78307


**Validar a distância entre os endereços**

In [216]:
import pandas as pd
import numpy as np

# Função para calcular a distância entre duas coordenadas (latitude e longitude) usando a fórmula de Haversine
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Raio da Terra em quilômetros
    phi1 = np.radians(lat1)
    phi2 = np.radians(lat2)
    delta_phi = np.radians(lat2 - lat1)
    delta_lambda = np.radians(lon2 - lon1)
    
    a = np.sin(delta_phi / 2) ** 2 + np.cos(phi1) * np.cos(phi2) * np.sin(delta_lambda / 2) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    
    distance = R * c  # Distância em quilômetros
    return distance


In [217]:
df_clientes.columns

Index(['ID_Cliente', 'nome', 'sobrenome', 'cep_preferencia', 'latitude',
       'longitude'],
      dtype='object')

In [218]:
df_clinicas.columns

Index(['ID_Clinica', 'nome', 'cep', 'nota_pesquisa', 'valor_consulta',
       'latitude', 'longitude'],
      dtype='object')

In [219]:
# Lista para armazenar os resultados
resultados = []

# Para cada cliente, calcular a distância para cada clínica
for _, cliente in df_clientes.iterrows():
    cliente_id = cliente['ID_Cliente']
    cliente_nome = cliente['nome']
    lat_cliente = cliente['latitude']
    lon_cliente = cliente['longitude']
    cep_cliente = cliente['cep_preferencia']
    
    # Para cada clínica, calcular a distância
    for _, clinica in df_clinicas.iterrows():
        clinica_id = clinica['ID_Clinica']
        clinica_nome = clinica['nome']
        lat_clinica = clinica['latitude']
        lon_clinica = clinica['longitude']
        cep_clinica = clinica['cep']
        
        # Calcular a distância usando a fórmula de Haversine
        distancia = haversine(lat_cliente, lon_cliente, lat_clinica, lon_clinica)
        
        # Adicionar os resultados na lista
        resultados.append({
            'ID_Cliente':cliente_id,
            'Cliente': cliente_nome,
            'CEP_Cliente': cep_cliente,
            'ID_Clinica': clinica_id,
            'Clínica': clinica_nome,
            'CEP_Clinica':cep_clinica,
            'Distância (km)': distancia
        })


In [220]:

# Criar o DataFrame com os resultados
df_distancias = pd.DataFrame(resultados)

# Ordenar pelo cliente e pela distância
df_distancias_sorted = df_distancias.sort_values(by=['Cliente', 'Distância (km)'])

In [221]:
df_distancias_sorted

,ID_Cliente,Cliente,CEP_Cliente,ID_Clinica,Clínica,CEP_Clinica,Distância (km)
96,4,Ana,04773-120,4,Clínica Estrela da Saúde,04773-120,0.000000
94,4,Ana,04773-120,2,Clínica Médico e Família,04773-110,0.089844
113,4,Ana,04773-120,21,Clínica Viver Bem,04773-170,0.163275
103,4,Ana,04773-120,11,Clínica Saúde Vida,04773-140,0.187613
116,4,Ana,04773-120,24,Clínica Dor e Bem Estar,04773-180,0.229842
...,...,...,...,...,...,...,...
268,9,Rafael,04773-130,21,Clínica Viver Bem,04773-170,6983.584030
258,9,Rafael,04773-130,11,Clínica Saúde Vida,04773-140,6983.695594
251,9,Rafael,04773-130,4,Clínica Estrela da Saúde,04773-120,6983.698677
249,9,Rafael,04773-130,2,Clínica Médico e Família,04773-110,6983.701506


In [222]:
df_distancias_sorted.to_excel('distancia_cliente_clinica.xlsx', index=True)

**Transformar a distância em metros e kms no nosso padrão**

In [223]:
df_com_km = df_distancias_sorted.copy()
df_com_km['Distancia_real'] = df_com_km['Distância (km)'] * 1.852  # Conversão para km
df_com_km


,ID_Cliente,Cliente,CEP_Cliente,ID_Clinica,Clínica,CEP_Clinica,Distância (km),Distancia_real
96,4,Ana,04773-120,4,Clínica Estrela da Saúde,04773-120,0.000000,0.000000
94,4,Ana,04773-120,2,Clínica Médico e Família,04773-110,0.089844,0.166392
113,4,Ana,04773-120,21,Clínica Viver Bem,04773-170,0.163275,0.302386
103,4,Ana,04773-120,11,Clínica Saúde Vida,04773-140,0.187613,0.347459
116,4,Ana,04773-120,24,Clínica Dor e Bem Estar,04773-180,0.229842,0.425668
...,...,...,...,...,...,...,...,...
268,9,Rafael,04773-130,21,Clínica Viver Bem,04773-170,6983.584030,12933.597623
258,9,Rafael,04773-130,11,Clínica Saúde Vida,04773-140,6983.695594,12933.804240
251,9,Rafael,04773-130,4,Clínica Estrela da Saúde,04773-120,6983.698677,12933.809949
249,9,Rafael,04773-130,2,Clínica Médico e Família,04773-110,6983.701506,12933.815190


In [224]:
df_com_km.to_excel('distancia_cliente_clinica.xlsx', index=True)

**Pesquisa de satisfação**

In [225]:
df_clinicas.head()

,ID_Clinica,nome,cep,nota_pesquisa,valor_consulta,latitude,longitude
0,1,Clínica Saúde e Bem Estar,05868-130,4.2,150,-23.67145,-46.78343
1,2,Clínica Médico e Família,04773-110,3.8,120,-23.68298,-46.70859
2,3,Clínica Boa Vista,05860-020,4.5,180,-23.66574,-46.76681
3,4,Clínica Estrela da Saúde,04773-120,4.0,170,-23.68342,-46.70933
4,5,Clínica Vida Plena,05868-140,3.7,160,-23.67058,-46.78307


In [226]:
df_pesquisa_satisfacao = df_clinicas[['ID_Clinica','nota_pesquisa']]
df_pesquisa_satisfacao.head()

,ID_Clinica,nota_pesquisa
0,1,4.2
1,2,3.8
2,3,4.5
3,4,4.0
4,5,3.7


**Valor da consulta**

In [227]:
df_valor_consulta = df_clinicas[['ID_Clinica','valor_consulta']]
df_valor_consulta.head()

,ID_Clinica,valor_consulta
0,1,150
1,2,120
2,3,180
3,4,170
4,5,160


**Listar as clinicas com base na distância, pesquisa e valor**

In [228]:
# 1. Fazer merge dos DataFrames
df = df_com_km.merge(df_pesquisa_satisfacao, on='ID_Clinica', how='left') \
              .merge(df_valor_consulta, on='ID_Clinica', how='left')


In [229]:
df

,ID_Cliente,Cliente,CEP_Cliente,ID_Clinica,Clínica,CEP_Clinica,Distância (km),Distancia_real,nota_pesquisa,valor_consulta
0,4,Ana,04773-120,4,Clínica Estrela da Saúde,04773-120,0.000000,0.000000,4.0,170
1,4,Ana,04773-120,2,Clínica Médico e Família,04773-110,0.089844,0.166392,3.8,120
2,4,Ana,04773-120,21,Clínica Viver Bem,04773-170,0.163275,0.302386,4.4,200
3,4,Ana,04773-120,11,Clínica Saúde Vida,04773-140,0.187613,0.347459,4.2,175
4,4,Ana,04773-120,24,Clínica Dor e Bem Estar,04773-180,0.229842,0.425668,3.6,140
...,...,...,...,...,...,...,...,...,...,...
336,9,Rafael,04773-130,21,Clínica Viver Bem,04773-170,6983.584030,12933.597623,4.4,200
337,9,Rafael,04773-130,11,Clínica Saúde Vida,04773-140,6983.695594,12933.804240,4.2,175
338,9,Rafael,04773-130,4,Clínica Estrela da Saúde,04773-120,6983.698677,12933.809949,4.0,170
339,9,Rafael,04773-130,2,Clínica Médico e Família,04773-110,6983.701506,12933.815190,3.8,120


In [230]:
# 2. Filtrar as clínicas com nota de pesquisa maior que 3
df_filtrado = df[df['nota_pesquisa'] > 3]

In [231]:
df_filtrado

,ID_Cliente,Cliente,CEP_Cliente,ID_Clinica,Clínica,CEP_Clinica,Distância (km),Distancia_real,nota_pesquisa,valor_consulta
0,4,Ana,04773-120,4,Clínica Estrela da Saúde,04773-120,0.000000,0.000000,4.0,170
1,4,Ana,04773-120,2,Clínica Médico e Família,04773-110,0.089844,0.166392,3.8,120
2,4,Ana,04773-120,21,Clínica Viver Bem,04773-170,0.163275,0.302386,4.4,200
3,4,Ana,04773-120,11,Clínica Saúde Vida,04773-140,0.187613,0.347459,4.2,175
4,4,Ana,04773-120,24,Clínica Dor e Bem Estar,04773-180,0.229842,0.425668,3.6,140
...,...,...,...,...,...,...,...,...,...,...
336,9,Rafael,04773-130,21,Clínica Viver Bem,04773-170,6983.584030,12933.597623,4.4,200
337,9,Rafael,04773-130,11,Clínica Saúde Vida,04773-140,6983.695594,12933.804240,4.2,175
338,9,Rafael,04773-130,4,Clínica Estrela da Saúde,04773-120,6983.698677,12933.809949,4.0,170
339,9,Rafael,04773-130,2,Clínica Médico e Família,04773-110,6983.701506,12933.815190,3.8,120


In [232]:
# 3. Criar uma pontuação combinada (menor distância, maior nota e menor valor de consulta)
# Podemos normalizar a distância, nota de pesquisa e valor de consulta para que fiquem na mesma escala
df_filtrado['distancia_normalizada'] = df_filtrado['Distancia_real'] / df_filtrado['Distancia_real'].max()
df_filtrado['nota_normalizada'] = df_filtrado['nota_pesquisa'] / 5.0  # Max nota é 5
df_filtrado['valor_normalizado'] = df_filtrado['valor_consulta'] / df_filtrado['valor_consulta'].max()


In [233]:
df_filtrado.head()

,ID_Cliente,Cliente,CEP_Cliente,ID_Clinica,Clínica,CEP_Clinica,Distância (km),Distancia_real,nota_pesquisa,valor_consulta,distancia_normalizada,nota_normalizada,valor_normalizado
0,4,Ana,04773-120,4,Clínica Estrela da Saúde,04773-120,0.000000,0.000000,4.0,170,0.000000,0.80,0.809524
1,4,Ana,04773-120,2,Clínica Médico e Família,04773-110,0.089844,0.166392,3.8,120,0.000010,0.76,0.571429
2,4,Ana,04773-120,21,Clínica Viver Bem,04773-170,0.163275,0.302386,4.4,200,0.000018,0.88,0.952381
3,4,Ana,04773-120,11,Clínica Saúde Vida,04773-140,0.187613,0.347459,4.2,175,0.000021,0.84,0.833333
4,4,Ana,04773-120,24,Clínica Dor e Bem Estar,04773-180,0.229842,0.425668,3.6,140,0.000026,0.72,0.666667


In [234]:
# 4. Criar a pontuação
# Vamos dar mais peso para a distância e a nota (supondo que a distância seja mais importante que o valor da consulta)
df_filtrado['pontuacao'] = (1 - df_filtrado['distancia_normalizada']) * 0.4 + df_filtrado['nota_normalizada'] * 0.4 + (1 - df_filtrado['valor_normalizado']) * 0.2


In [235]:
df_filtrado.head()

,ID_Cliente,Cliente,CEP_Cliente,ID_Clinica,Clínica,CEP_Clinica,Distância (km),Distancia_real,nota_pesquisa,valor_consulta,distancia_normalizada,nota_normalizada,valor_normalizado,pontuacao
0,4,Ana,04773-120,4,Clínica Estrela da Saúde,04773-120,0.000000,0.000000,4.0,170,0.000000,0.80,0.809524,0.758095
1,4,Ana,04773-120,2,Clínica Médico e Família,04773-110,0.089844,0.166392,3.8,120,0.000010,0.76,0.571429,0.789710
2,4,Ana,04773-120,21,Clínica Viver Bem,04773-170,0.163275,0.302386,4.4,200,0.000018,0.88,0.952381,0.761517
3,4,Ana,04773-120,11,Clínica Saúde Vida,04773-140,0.187613,0.347459,4.2,175,0.000021,0.84,0.833333,0.769325
4,4,Ana,04773-120,24,Clínica Dor e Bem Estar,04773-180,0.229842,0.425668,3.6,140,0.000026,0.72,0.666667,0.754656


In [236]:
# 5. Ordenar pelo maior score
df_filtrado = df_filtrado.sort_values(by='pontuacao', ascending=False)


In [237]:
df_filtrado.head()

,ID_Cliente,Cliente,CEP_Cliente,ID_Clinica,Clínica,CEP_Clinica,Distância (km),Distancia_real,nota_pesquisa,valor_consulta,distancia_normalizada,nota_normalizada,valor_normalizado,pontuacao
124,1,João,05868-130,1,Clínica Saúde e Bem Estar,05868-130,0.000000,0.000000,4.2,150,0.000000,0.84,0.714286,0.793143
280,5,Pedro,05868-140,1,Clínica Saúde e Bem Estar,05868-130,0.103454,0.191596,4.2,150,0.000012,0.84,0.714286,0.793138
191,7,Lucas,05868-150,1,Clínica Saúde e Bem Estar,05868-130,0.201351,0.372903,4.2,150,0.000022,0.84,0.714286,0.793134
98,10,Fernanda,05868-150,1,Clínica Saúde e Bem Estar,05868-130,0.201351,0.372903,4.2,150,0.000022,0.84,0.714286,0.793134
50,3,Carlos,05860-020,1,Clínica Saúde e Bem Estar,05868-130,1.807773,3.347995,4.2,150,0.000201,0.84,0.714286,0.793062


In [238]:
# # 6. Adicionar um índice para facilitar a visualização da ordem
# df_filtrado['rank'] = df_filtrado['pontuacao'].rank(ascending=False).astype(int)


In [239]:
df_filtrado['rank_novo'] = df_filtrado.groupby('ID_Cliente')['pontuacao'].rank(ascending=False, method='first').astype(int)

In [240]:
df_filtrado.head()

,ID_Cliente,Cliente,CEP_Cliente,ID_Clinica,Clínica,CEP_Clinica,Distância (km),Distancia_real,nota_pesquisa,valor_consulta,distancia_normalizada,nota_normalizada,valor_normalizado,pontuacao,rank_novo
124,1,João,05868-130,1,Clínica Saúde e Bem Estar,05868-130,0.000000,0.000000,4.2,150,0.000000,0.84,0.714286,0.793143,1
280,5,Pedro,05868-140,1,Clínica Saúde e Bem Estar,05868-130,0.103454,0.191596,4.2,150,0.000012,0.84,0.714286,0.793138,1
191,7,Lucas,05868-150,1,Clínica Saúde e Bem Estar,05868-130,0.201351,0.372903,4.2,150,0.000022,0.84,0.714286,0.793134,1
98,10,Fernanda,05868-150,1,Clínica Saúde e Bem Estar,05868-130,0.201351,0.372903,4.2,150,0.000022,0.84,0.714286,0.793134,1
50,3,Carlos,05860-020,1,Clínica Saúde e Bem Estar,05868-130,1.807773,3.347995,4.2,150,0.000201,0.84,0.714286,0.793062,1


In [241]:
df = df_filtrado
df['StatusClinica'] = ''
df['Data_Acao'] = ''
df['Hora_Acao'] = ''

df.to_excel('tabela_ordem_sugestao.xlsx', index=False)

In [242]:

# import datetime

# tempo_atual = datetime.datetime.now()
# print(f"Iniciando processo às {tempo_atual.strftime('%H:%M:%S')}")
# tempo_decorrido = 0

# while True:

#     agora = datetime.datetime.now()
#     tempo_processamento = (agora - tempo_atual)
#     tempo_decorrido = int(tempo_processamento.total_seconds())
#     #tempo_inteiro = int(tempo_segundos)
  
#     if tempo_decorrido > 5:
#         hora_formatada = agora.strftime('%H:%M:%S')
#         print(f'Quando o processo parou: {hora_formatada}', f'Tempo de duração do processo: {tempo_decorrido}')
#         break

In [244]:
df.head()

,ID_Cliente,Cliente,CEP_Cliente,ID_Clinica,Clínica,CEP_Clinica,Distância (km),Distancia_real,nota_pesquisa,valor_consulta,distancia_normalizada,nota_normalizada,valor_normalizado,pontuacao,rank_novo,StatusClinica,Data_Acao,Hora_Acao
124,1,João,05868-130,1,Clínica Saúde e Bem Estar,05868-130,0.000000,0.000000,4.2,150,0.000000,0.84,0.714286,0.793143,1,,,
280,5,Pedro,05868-140,1,Clínica Saúde e Bem Estar,05868-130,0.103454,0.191596,4.2,150,0.000012,0.84,0.714286,0.793138,1,,,
191,7,Lucas,05868-150,1,Clínica Saúde e Bem Estar,05868-130,0.201351,0.372903,4.2,150,0.000022,0.84,0.714286,0.793134,1,,,
98,10,Fernanda,05868-150,1,Clínica Saúde e Bem Estar,05868-130,0.201351,0.372903,4.2,150,0.000022,0.84,0.714286,0.793134,1,,,
50,3,Carlos,05860-020,1,Clínica Saúde e Bem Estar,05868-130,1.807773,3.347995,4.2,150,0.000201,0.84,0.714286,0.793062,1,,,
